In [1]:
import tensorflow as tf
import autokeras as ak
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", physical_devices)


Num GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
data = pd.read_csv(r'data\train.csv')
test = pd.read_csv(r'data\test.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data\\train.csv'

In [6]:
X = data.drop(columns=['Survived','PassengerId'])  # Remove the target column
y = data['Survived']  # Extract the target column

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [7]:
# Initialize the AutoKeras Structured Data Classifier with column details
clf = ak.StructuredDataClassifier(

    max_trials=15
)

# Fit the model using extracted X and y
clf.fit(X_train, y_train)
model=clf.export_model()
model.save('models/ak_titanic_kaggle', save_format='tf')

INFO:tensorflow:Reloading Oracle from existing project .\structured_data_classifier\oracle.json
INFO:tensorflow:Reloading Tuner from .\structured_data_classifier\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
19/19 [==============================] - 4s 7ms/step - loss: 0.6623 - accuracy: 0.6342
Epoch 2/5
19/19 [==============================] - 0s 7ms/step - loss: 0.5874 - accuracy: 0.6829
Epoch 3/5
19/19 [==============================] - 0s 7ms/step - loss: 0.5349 - accuracy: 0.7349
Epoch 4/5
19/19 [==============================] - 0s 7ms/step - loss: 0.4974 - accuracy: 0.7869
Epoch 5/5
19/19 [==============================] - 0s 6ms/step - loss: 0.4711 - accuracy: 0.8054
INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets
INFO:tensorflow:Assets written to: models/ak_titanic_kaggle\assets


In [8]:
predicted_y = clf.predict(X_test)
test_loss, test_acc = clf.evaluate(X_test,
                                    y_test,
                                    verbose=0) 
print(test_loss, test_acc)

#predicted_y

10/10 [==============================] - 0s 4ms/step
0.4789118468761444 0.803389847278595


In [23]:
predicted_y = clf.predict(test.drop(columns=['PassengerId']))

14/14 [==============================] - 0s 3ms/step


In [24]:
sub = pd.merge(test[['PassengerId']], pd.DataFrame(predicted_y), left_index=True, right_index=True)#.to_csv('data/ak_submission.csv', index=False)

In [25]:
sub

,PassengerId,0
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [13]:
sub

In [26]:
sub.to_csv('data/sub.csv', index=False)